In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from MOMStrategy import MOMStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, window, num, rev):
    window = int(window)
    num = int(num)
    config['title'] = "MOMStrategy" + "_" + str(window) + "_" + str(num) + '_' + str(rev)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    portfolio_handler = PortfolioHandler(
        config, data_handler, events_queue
    )
    strategy = MOMStrategy(config, events_queue, data_handler, portfolio_handler,
                         window = window, num = num, rev = rev)  

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler, portfolio_handler = portfolio_handler)

    results = backtest.start_trading()

    # dict_ans = {
    #     "window": [window],
    #     "num": [num],
    #     "rev": [rev], 
    #     "Sharpe Ratio": [results['sharpe']],
    #     "Total Returns": [(results['cum_returns'][-1] - 1)],
    #     "Max Drawdown": [(results["max_drawdown"] * 100.0)],
    #     "Max Drawdown Duration": [(results['max_drawdown_duration'])],
    #     "Trades": [results['trade_info']['trading_num']],
    #     "Trade Winning": [results['trade_info']['win_pct']],
    #     "Average Trade": [results['trade_info']['avg_trd_pct']],
    #     "Average Win": [results['trade_info']['avg_win_pct']],
    #     "Average Loss": [results['trade_info']['avg_loss_pct']],
    #     "Best Trade": [results['trade_info']['max_win_pct']],
    #     "Worst Trade": [results['trade_info']['max_loss_pct']],
    #     "Worst Trade Date": [results['trade_info']['max_loss_dt']],
    #     "Avg Days in Trade": [results['trade_info']['avg_dit']]
    # }
    # return pd.DataFrame(dict_ans)
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MOMStrategy",
    "title": "MOMStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2018-01-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq="60" + "T"),
    "equity": 1.0,
    "freq": 60,  # min
    "commission_ratio": 0.002,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT', 'CMTBNB', 'CMTBTC', 'CMTETH', 
                'ETHUSDT', 'LTCUSDT', 'VENBNB', #  'EOSUSDT' 'XRPUSDT'
               'VENBTC', 'VENETH']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'window': (1, 120),
    'num': (2, 5)},
    is_int = [1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data,
        'rev': 1
    },
    random_state = 1
)

BO.maximize(init_points=20, n_iter=40, acq="ei", xi=0.01, **gp_params)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |       num |    window | 
---------------------------------
MOMStrategy_38_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time 

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.5493344762
Sortino Ratio: -4.1916345120
Max Drawdown: 0.7022306118
Max Drawdown Duration: 199700
Total Returns: -0.5369790302
Annualized Returns: -2.1777482890
Compound Annual Growth Rate: -0.9559622725
---------------------------------
Trades: 670
Trade Winning: 59.55%
Average Trade: 0.09%
Average Win: 2.76%
Average Loss: -3.83%
Best Trade: 28.15%
Worst Trade: -40.28%
Worst Trade Date: 7 days 03:00:00
Avg Days in Trade: 0 days 12:50:41
---------------------------------
    6 | 00m24s |   -0.53698 | 4.000000 | 76.000000 | 
---------------------------------
MOMStrategy_6_4_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-0

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -8.3728352015
Sortino Ratio: -15.6265113705
Max Drawdown: 0.9507526322
Max Drawdown Duration: 164700
Total Returns: -0.9273056300
Annualized Returns: -3.7607394996
Compound Annual Growth Rate: -0.9999758591
---------------------------------
Trades: 5147
Trade Winning: 58.01%
Average Trade: 0.21%
Average Win: 1.48%
Average Loss: -1.53%
Best Trade: 16.61%
Worst Trade: -23.74%
Worst Trade Date: 0 days 04:00:00
Avg Days in Trade: 0 days 01:40:41
---------------------------------
   11 | 00m10s |   -0.92731 | 4.000000 | 2.000000 | 
---------------------------------
MOMStrategy_77_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -4.0807014179
Sortino Ratio: -6.1410750937
Max Drawdown: 0.8155252749
Max Drawdown Duration: 205900
Total Returns: -0.7572912820
Annualized Returns: -3.0712368661
Compound Annual Growth Rate: -0.9967924046
---------------------------------
Trades: 1044
Trade Winning: 57.85%
Average Trade: 0.10%
Average Win: 2.26%
Average Loss: -2.85%
Best Trade: 20.72%
Worst Trade: -50.32%
Worst Trade Date: 2 days 03:00:00
Avg Days in Trade: 0 days 06:08:23
---------------------------------
   16 | 00m15s |   -0.75729 | 3.000000 | 26.000000 | 
---------------------------------
MOMStrategy_51_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.7455271449
Sortino Ratio: -4.1979895733
Max Drawdown: 0.7095354933
Max Drawdown Duration: 199100
Total Returns: -0.5244700103
Annualized Returns: -2.1270172641
Compound Annual Growth Rate: -0.9509344033
---------------------------------
Trades: 482
Trade Winning: 57.47%
Average Trade: 0.04%
Average Win: 3.36%
Average Loss: -4.45%
Best Trade: 41.59%
Worst Trade: -45.77%
Worst Trade Date: 13 days 17:00:00
Avg Days in Trade: 0 days 21:31:00
---------------------------------
   21 | 00m37s |   -0.52447 | 5.000000 | 120.000000 | 
---------------------------------
MOMStrategy_57_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-0

...

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -3.0229968320
Sortino Ratio: -4.6163480994
Max Drawdown: 0.8165885827
Max Drawdown Duration: 205900
Total Returns: -0.7475848049
Annualized Returns: -3.0318717086
Compound Annual Growth Rate: -0.9962394950
---------------------------------
Trades: 798
Trade Winning: 55.89%
Average Trade: 0.18%
Average Win: 2.37%
Average Loss: -2.58%
Best Trade: 21.85%
Worst Trade: -49.71%
Worst Trade Date: 2 days 01:00:00
Avg Days in Trade: 0 days 05:20:54
---------------------------------
   36 | 00m20s |   -0.74758 | 2.000000 | 31.000000 | 
---------------------------------
MOMStrategy_32_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.8816538076
Sortino Ratio: -1.4632231859
Max Drawdown: 0.6399541953
Max Drawdown Duration: 168600
Total Returns: -0.3946337318
Annualized Returns: -1.6004590235
Compound Annual Growth Rate: -0.8693941072
---------------------------------
Trades: 376
Trade Winning: 58.78%
Average Trade: 0.30%
Average Win: 2.68%
Average Loss: -3.10%
Best Trade: 15.96%
Worst Trade: -36.50%
Worst Trade Date: 7 days 07:00:00
Avg Days in Trade: 0 days 11:13:24
---------------------------------
   41 | 00m37s |   -0.39463 | 2.000000 | 118.000000 | 
---------------------------------
MOMStrategy_22_4_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.1939324111
Sortino Ratio: -3.4860105640
Max Drawdown: 0.6838326331
Max Drawdown Duration: 205900
Total Returns: -0.4990630917
Annualized Returns: -2.0239780942
Compound Annual Growth Rate: -0.9394026819
---------------------------------
Trades: 898
Trade Winning: 59.80%
Average Trade: 0.17%
Average Win: 2.77%
Average Loss: -3.69%
Best Trade: 30.48%
Worst Trade: -41.54%
Worst Trade Date: 3 days 00:00:00
Avg Days in Trade: 0 days 11:49:03
---------------------------------
   46 | 00m23s |   -0.49906 | 5.000000 | 47.000000 | 
---------------------------------
MOMStrategy_45_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.7163184834
Sortino Ratio: -4.2002971438
Max Drawdown: 0.7996073018
Max Drawdown Duration: 205400
Total Returns: -0.7045043864
Annualized Returns: -2.8571566782
Compound Annual Growth Rate: -0.9928749105
---------------------------------
Trades: 409
Trade Winning: 57.21%
Average Trade: -0.08%
Average Win: 2.43%
Average Loss: -3.43%
Best Trade: 25.54%
Worst Trade: -33.36%
Worst Trade Date: 6 days 07:00:00
Avg Days in Trade: 0 days 10:33:09
---------------------------------
   51 | 00m29s |   -0.70450 | 2.000000 | 86.000000 | 
---------------------------------
MOMStrategy_8_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.7996991931
Sortino Ratio: -4.6136495283
Max Drawdown: 0.8101880762
Max Drawdown Duration: 204900
Total Returns: -0.6970519150
Annualized Returns: -2.8269327665
Compound Annual Growth Rate: -0.9921175728
---------------------------------
Trades: 427
Trade Winning: 57.61%
Average Trade: -0.06%
Average Win: 2.43%
Average Loss: -3.44%
Best Trade: 15.96%
Worst Trade: -31.89%
Worst Trade Date: 5 days 18:00:00
Avg Days in Trade: 0 days 10:06:36
---------------------------------
   56 | 00m27s |   -0.69705 | 2.000000 | 82.000000 | Warning: Test point chose at random due to repeated sample.

---------------------------------
MOMStrategy_8_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CM

In [7]:
print(BO.res['max'])

{'max_val': -0.29874592668087796, 'max_params': {'window': 120.0, 'num': 2.0}}


In [8]:
BO.maximize(init_points=0, n_iter=40, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |       num |    window | 
---------------------------------
MOMStrategy_39_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Dat

---------------------------------
MOMStrategy_89_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

---------------------------------
MOMStrategy_84_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.23402219e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
MOMStrategy_15_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

...

---------------------------------
MOMStrategy_91_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

---------------------------------
MOMStrategy_12_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

---------------------------------
MOMStrategy_30_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

---------------------------------
MOMStrategy_27_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Run

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.07858345e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [9]:
print(BO.res['max'])

{'max_val': -0.29874592668087796, 'max_params': {'window': 120.0, 'num': 2.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
24,"[18.0, 2.0]",-0.934106
10,"[2.0, 4.0]",-0.927306
97,"[22.0, 2.0]",-0.917399
52,"[20.0, 2.0]",-0.874129
22,"[4.0, 2.0]",-0.867321
19,"[19.0, 3.0]",-0.851938
75,"[15.0, 3.0]",-0.845026
79,"[22.0, 3.0]",-0.843263
89,"[12.0, 3.0]",-0.840269
73,"[30.0, 2.0]",-0.835041


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)